# QUANTILES ET SUPERQUANTILES

*Théophile&nbsp;BARANGER, Gauthier&nbsp;DECULTOT, Exaucé&nbsp;ADJIM LUWEH NGARTI, Omar&nbsp;KHATTAB*
- - -

Dans ce notebook se trouvent les codes correspondant aux simulations abordées dans le rapport concernant le projet "Quantiles et superquantiles", dont le sujet (composé par [Bernard Bercu](https://bernardbercu.files.wordpress.com/)) est disponible ici&nbsp;:&nbsp;[Lien vers le sujet](https://bernardbercu.files.wordpress.com/2021/01/projet-quantiles-2021.pdf).

In [ ]:
from matplotlib.pyplot import *
from math import *
from numpy import *
from numpy.random import *
from time import *

## Fonctions utiles
On commence par construire les fonctions qui nous permettront de simuler des variables aléatoires de lois bien connues, qui nous permettrons de tester nos estimateurs. On se place dans un cadre où les lois seront à densité, de fonction de répartition continue strictement croissante, et donc on utilise naturellement la méthode d'inversion pour les simuler.

In [ ]:
def pareto(n=10000,xm_k=[1,3]):
    return xm_k[0]*((1-rand(n))**(-1/xm_k[1]))

In [ ]:
def qpareto(xm_k=[1,3],alpha=0.5):
    return (xm_k[0]*(1-alpha)**(-1/xm_k[1]))

In [ ]:
def dpareto(x,xm_k):
    return (xm_k[1]*(xm_k[0]**(xm_k[1])))/x**(xm_k[1]+1)

In [ ]:
def sqpareto(xm_k=[1,3],alpha=0.5):
    return ((xm_k[1]*xm_k[0])/(xm_k[1]-1))*((1-alpha)**(-1/xm_k[1]))

In [ ]:
def tpareto(xm_k=[1,3],alpha=0.5):
    return((xm_k[1]*xm_k[0])/(alpha*(xm_k[1]-1)))*(1-(1-alpha)**(1-(1/xm_k[1])))

In [ ]:
def expo(n=10000,lamb=[1]):
    return((-1/lamb[0])*log(1-rand(n)))   

In [ ]:
def qexpo(lamb=[1],alpha=0.5):
    return((-1/lamb[0])*log(1-alpha)) 

In [ ]:
def dexpo(x,lamb):
    return lamb[0]*exp(-lamb[0]*x)

In [ ]:
def sqexpo(lamb=[1],alpha=0.5):
    return (1/lamb[0])*(1-log(1-alpha)) 

In [ ]:
def texpo(lamb=[1],alpha=0.5):
    return (1/(alpha*lamb[0]))*((1-alpha)*log(1-alpha)+alpha)

## Estimateurs
On définit ensuite les estimateurs qu'il s'agira de tester ensuite par simulation.

### Estimation de quantiles
On définit les estimateurs <code>theta_bar</code> et <code>theta_hat</code> pour l'estimation de quantiles, et qui correspondent respectivement aux estimateurs $\bar{\theta}_n$ et $\widehat{\theta}_n$ proposés dans le sujet. Ici, on construit directement les suites $(\bar{\theta}_n)$ et $(\widehat{\theta}_n)$ sur la base d'un échantillon $X$ et du paramètre $\alpha$. Comme il est précisé dans le code, nous avons défini le pas de l'algorithme de Robbins-Monro comme $a_n=\frac{1}{n+1}$.

In [ ]:
def theta_bar(alpha,X):
    T=[]
    for n in range(1, len(X)+1):
        Y=X[:n]
        Y.sort()
        T.append(Y[math.floor(n*alpha)])
    return T

In [ ]:
def theta_hat(alpha,X):
    T=np.zeros(len(X))
    # Initialisation theta_hat_0 = X[0]
    T[0]=X[0]
    # Construction de la suite des theta_hat_n
    for i in range(1, len(X)):
        # Robbins-Monro avec pas en 1/(i+1)
        T[i] = T[i-1] - (1/i)*(int(X[i]<=T[i-1])-alpha)
    return T

### Estimation de superquantiles

In [ ]:
def theta_ronde_bar(alpha,X):
    T=[]
    for n in range(1, len(X)+1):
        Y=X[:n]
        Y.sort()
        T.append((1/((1-alpha)*n))*np.sum(Y[math.floor(n*alpha):]))
    return T

In [ ]:
def theta_ronde_hat(alpha,X):
    Theta=np.zeros(len(X))
    Theta_ronde=np.zeros(len(X))
    # Initialisation theta_hat_0 = X[0]
    Theta[0]=X[0]
    Theta_ronde[0]=X[0]
    # Construction de la suite des theta_ronde_n
    for i in range(1, len(X)):
        # Robbins-Monro avec pas en 1/(i+1)
        Theta[i] = Theta[i-1] - (1/i)*(int(X[i]<=Theta[i-1])-alpha)
        Theta_ronde[i] = Theta_ronde[i-1] + (1/i)*((X[i]/(1-alpha))*int(X[i]>Theta[i-1])-Theta_ronde[i-1])
    return Theta_ronde

### Dépréciation moyenne

In [ ]:
def tau_bar(alpha,X):
    T=[]
    Tau=[]
    for n in range(1, len(X)+1):
        Y=X[:n]
        Y.sort()
        T.append((1/((1-alpha)*n))*np.sum(Y[math.floor(n*alpha):]))
        Tau.append((1/alpha)*((1/n)*np.sum(Y)-(1-alpha)*T[n-1]))
    return Tau

In [ ]:
def tau_hat(alpha,X):
    T=np.zeros(len(X))
    # Initialisation tau_hat_0 = X0
    T[0]=X[0]
    Tau=np.zeros(len(X))
    Tau[0]=X[0]
    c=2/3

    # Construction de la suite des tau_hat_n
    for i in range(1, len(X)):
        # Robbins-Monro avec pas en 1/(i**c)
        T[i] = T[i-1] - (1/i)*(int(X[i]<=T[i-1])-alpha)
        bi=1/(i**c)
        Tau[i]=Tau[i-1]+bi*((X[i]/alpha)*(int(X[i] < T[i-1]))-Tau[i-1]+(1/alpha)*(-1 + 1/(bi*(i+1)))*(X[i]-np.mean(X[0:i])))
    return Tau

## Fonctions de convergence

### Convergence presque sûre

In [ ]:
def convergence_ps_quantile(N,Nr,alpha,loi,param,method,display=True):
    if display:
        figure(figsize=(20,10))
    #en fonction de la methode on prend la fonction a afficher
    if method.__name__.startswith('theta_ronde')==True:
        t_alpha=eval('sq'+loi)(param,alpha)
    elif method.__name__.startswith('tau')==True:
        t_alpha=eval('t'+loi)(param,alpha)
    else:   
        t_alpha=eval('q'+loi)(param,alpha)
    toc = process_time()
    #boucle pour plusieurs réalisations
    for i in range(Nr):
        X=eval(loi)(N,param)
        T=method(alpha,X)
        if display:
            plot(np.arange(N),T)
    tic = process_time()
    if display:
        hlines(t_alpha, 0, N, 'k', 'dashed', lw=5)
        if method==theta_bar:
            title(r"Convergence presque sûre de "
              r"$\bar{\theta}_n$",
              fontsize=20)
        elif method==theta_hat:
            title(r"Convergence presque sûre de "
              r"$\widehat{\theta}_n$",
              fontsize=20)
        elif method==theta_ronde_bar:
            title(r"Convergence presque sûre de "
              r"$\bar{\vartheta}_n$",
              fontsize=20)
        elif method==theta_ronde_hat:
            title(r"Convergence presque sûre de "
              r"$\widehat{\vartheta}_n$",
              fontsize=20)
        elif method==tau_hat:
            title(r"Convergence presque sûre de "
              r"$\widehat{\tau}_n$",
              fontsize=20)
        else:
            title(r"Convergence presque sûre de "
              r"$\bar{\tau}_n$",
              fontsize=20)
        tight_layout()
        subplots_adjust(top=0.88)
        #f_name='Cv_'+method.__name__+'_'+loi+'.png'
        #savefig(f_name, bbox_inches='tight', format="png")
    return tic-toc

### Normalité asymptotique

In [ ]:
def asymptotic_variance(method,alpha,loi,param,estimateur="quantile"):
    if estimateur=="quantile":
        t_alpha=eval('q'+loi)(param,alpha)
        d_talpha=eval('d'+loi)(t_alpha,param)
    
        if (method==theta_hat and d_talpha<=1/2):
            print("Attention, f(t_alpha) <= 1/2, la méthode 'theta_bar' n\'est pas valide.\n")
            return 0

        if method==theta_bar:
            return (alpha*(1-alpha))/(d_talpha)**2;
        elif method==theta_hat:
            return (alpha*(1-alpha))/(2*d_talpha-1);
            
    elif estimateur=="superquantile":
        if loi=='expo':
            return (1+alpha)/((1-alpha)*param[0]**2)
        elif loi=='pareto':
            return ((param[0]**2)/(param[1]-1))*((1-alpha)**(-1-(2/param[1])))*((2/(param[1]-2))-((1-alpha)/(param[1]-1)))
        else:
            print("Variance de l'estimateur superquantile inconnu pour cette loi.\n")
            return 0  
    else:
        return 0

In [ ]:
def normalite_as_quantile(N,Nr,alpha,loi,param,method,estimateur="quantile"):
    Z=[]
    if estimateur=="quantile":
        target=eval('q'+loi)(param,alpha)
    elif estimateur=="superquantile":
        target=eval('sq'+loi)(param,alpha)
    
    var=asymptotic_variance(method,alpha,loi,param,estimateur)
    if (var<=0):
        print("Erreur dans le calcul de la variance.\n")
        return
    
    figure(figsize=(20,10))
    for i in range(Nr):
        X=eval(loi)(N,param)
        T=method(alpha,X)
        Z.append((sqrt(N)/(sqrt(var)))*(T[N-1]-target))
    
    
    #Création de l'histogramme
    hist(Z,bins=linspace(-4,4,41),density=True);
    x=linspace(-4,4,100);
    #Ajout de la densité de la normale
    plot(x,exp(-x**2/2)/sqrt(2*pi),lw=3,color= "r");
    if method==theta_bar:
        title(r"Normalité asymptotique de "
        r"$\bar{\theta}_n$",
        fontsize=20)
    elif method==theta_hat:
        title(r"Normalité asymptotique de "
        r"$\widehat{\theta}_n$",
        fontsize=20)
    elif method==theta_ronde_bar:
        title(r"Normalité asymptotique de "
        r"$\bar{\vartheta}_n$",
        fontsize=20)
    elif method==theta_ronde_hat:
        title(r"Normalité asymptotique de "
        r"$\widehat{\vartheta}_n$",
        fontsize=20)
    elif method==tau_hat:
        title(r"Normalité asymptotique de "
        r"$\widehat{\tau}_n$",
        fontsize=20)
    else:
        title(r"Normalité asymptotique de "
        r"$\bar{\tau}_n$",
        fontsize=20)
    tight_layout()
    subplots_adjust(top=0.88)
    #f_name='Norm_'+method.__name__+'_'+loi+'.png'
    #savefig(f_name, bbox_inches='tight', format="png")

## Simulations

Voici les appels aux fonctions définies précedemment que nous avons fait pour générer les figures se trouvant dans le rapport.

In [ ]:
N=10**4
NR=10**4
M=20
alpha=0.5

### Quantile

In [ ]:
tps = convergence_ps_quantile(N,M,alpha,'expo',[2],theta_bar)
print("Temps d'exécution : {}s".format(round(tps,2)))

In [ ]:
tps = convergence_ps_quantile(N,M,alpha,'expo',[2],theta_hat)
print("Temps d'exécution : {}s".format(round(tps,2)))

In [ ]:
tps = convergence_ps_quantile(N,M,alpha,'pareto',[1,3],theta_bar)
print("Temps d'exécution : {}s".format(round(tps,2)))

In [ ]:
tps = convergence_ps_quantile(N,M,alpha,'pareto',[1,3],theta_hat)
print("Temps d'exécution : {}s".format(round(tps,2)))

In [ ]:
toc=process_time()
normalite_as_quantile(int(N/10),NR,alpha,'pareto',[1,3],theta_bar)
tic = process_time()
print("Temps d'exécution : {}s".format(round(tic-toc,2)))

In [ ]:
toc=process_time()
normalite_as_quantile(int(N/10),NR,alpha,'pareto',[1,3],theta_hat)
tic = process_time()
print("Temps d'exécution : {}s".format(round(tic-toc,2)))

In [ ]:
toc=process_time()
normalite_as_quantile(int(N/10),NR,alpha,'expo',[2],theta_bar)
tic = process_time()
print("Temps d'exécution : {}s".format(round(tic-toc,2)))

In [ ]:
toc=process_time()
normalite_as_quantile(int(N/10),NR,alpha,'expo',[2],theta_hat)
tic = process_time()
print("Temps d'exécution : {}s".format(round(tic-toc,2)))

### Superquantiles

In [ ]:
tps = convergence_ps_quantile(N,M,alpha,'expo',[2],theta_ronde_bar)
print("Temps d'exécution : {}s".format(round(tps,2)))

In [ ]:
tps = convergence_ps_quantile(N,M,alpha,'expo',[2],theta_ronde_hat)
print("Temps d'exécution : {}s".format(round(tps,2)))

In [ ]:
tps = convergence_ps_quantile(N,M,alpha,'pareto',[1,3],theta_ronde_bar)
print("Temps d'exécution : {}s".format(round(tps,2)))

In [ ]:
tps = convergence_ps_quantile(N,M,alpha,'pareto',[1,3],theta_ronde_hat)
print("Temps d'exécution : {}s".format(round(tps,2)))

In [ ]:
toc=process_time()
normalite_as_quantile(int(N/10),NR,alpha,'expo',[2],theta_ronde_hat,estimateur="superquantile")
tic = process_time()
print("Temps d'exécution : {}s".format(round(tic-toc,2)))

In [ ]:
toc=process_time()
normalite_as_quantile(int(N/10),NR,0.5,'pareto',[1,3],theta_ronde_hat,estimateur="superquantile")
tic = process_time()
print("Temps d'exécution : {}s".format(round(tic-toc,2)))

In [ ]:
toc=process_time()
normalite_as_quantile(int(N/10),NR,alpha,'pareto',[1,3],theta_ronde_bar,estimateur="superquantile")
tic = process_time()
print("Temps d'exécution : {}s".format(round(tic-toc,2)))

In [ ]:
toc=process_time()
normalite_as_quantile(int(N/10),NR,alpha,'expo',[2],theta_ronde_bar,estimateur="superquantile")
tic = process_time()
print("Temps d'exécution : {}s".format(round(tic-toc,2)))

### Dépréciation moyenne

In [ ]:
tps = convergence_ps_quantile(N,M,alpha,'expo',[2],tau_bar)
print("Temps d'exécution : {}s".format(round(tps,2)))

In [ ]:
tps = convergence_ps_quantile(N,M,alpha,'pareto',[1,3],tau_bar)
print("Temps d'exécution : {}s".format(round(tps,2)))

In [ ]:
tps = convergence_ps_quantile(2*N,M,alpha,'expo',[2],tau_hat)
print("Temps d'exécution : {}s".format(round(tps,2)))

In [ ]:
tps = convergence_ps_quantile(2*N,M,alpha,'pareto',[1,3],tau_hat)
print("Temps d'exécution : {}s".format(round(tps,2)))

## Temps de calcul des différents estimateurs

Le code suivant nous a permis de générer les tableaux se trouvant à la fin du rapport.

In [ ]:
q=arange(0.25,1,0.25)
p=[[1,3],[2,25],[3,50],[6,75],[10,100]]
U=[]
for k in range (0,5):    #5 itérations
    T=[]
    for i in range(0,len(p)):  #boucle sur les params
        a=[]
        for j in range(0,len(q)): #boucle sur les quantiles
            a.append(round(convergence_ps_quantile(N,M,q[j],'pareto',p[i],tau_hat,display=False),2))
        T.append(a)
    U.append(T)
U=np.array(U)
print( np.sum(U,axis=0)/len(U))   #moyenne